In [1]:
import os

import pandas as pd
import numpy as np

import torch


from modelscope import snapshot_download
from datasets import Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import get_peft_model, LoraConfig, TaskType



2024-03-13 21:52:36,364 - modelscope - INFO - PyTorch version 2.0.1 Found.
2024-03-13 21:52:36,366 - modelscope - INFO - Loading ast index from /root/.cache/modelscope/ast_indexer
2024-03-13 21:52:36,507 - modelscope - INFO - Loading done! Current index file version is 1.13.1, with md5 ce2a1413d67bf5615c2e26752330cf67 and a total number of 972 components indexed
/root/miniconda3/envs/translate/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2024-03-13 21:52:37,367] [INFO] [real_accelerator.py:191:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [2]:
REPO_DIRECTORY = r'/root/'
ABC_DICT_PATH = r'/autodl-tmp/AIST4010-Cantonese-Translator-Data/ABC-Dict/abc_dict.csv'

def load_abc_dataset():
    abc_dict = pd.read_csv(REPO_DIRECTORY + ABC_DICT_PATH)
    abc_dataset = Dataset.from_pandas(abc_dict)
    return abc_dataset

abc_dict_dataset = load_abc_dataset()
print(abc_dict_dataset[:10])

{'en': ['Scoop up water', 'Ladle out soup', 'Third son of a rich family', 'Young pigeon or squab that has been roasted', "Husband's second older brother", 'This time', 'One dollar and sixty cents', 'Rabbit', 'Exit through turnstile', 'Chop something into cubes'], 'yue': ['㧾水', '㧾湯', '三少', '乳鴿', '二少', '今勻', '個六', '兔仔', '出閘', '切粒']}


In [3]:
model_path=r'/root/autodl-tmp/01ai/Yi-6B-Chat-4bits'

# model = Model.from_pretrained('01ai/Yi-6B')

# model = AutoModelForCausalLM.from_pretrained(
#     model_name,
#     device_map="auto",
#     torch_dtype='auto'
# ).eval()


# tokenizer = AutoTokenizer.from_pretrained(model_name)

In [4]:
import torch
# from modelscope import snapshot_download, AutoModel, AutoTokenizer
import os
model_dir = snapshot_download('01ai/Yi-6B-Chat-4bits', cache_dir='/root/autodl-tmp', revision='master')

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=False)

# Since transformers 4.35.0, the GPT-Q/AWQ model can be loaded using AutoModelForCausalLM.
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    device_map="auto",
    torch_dtype='auto'
).eval()

# Prompt content: "hi"
messages = [
    {"role": "user", "content": "hi"}
]


input_ids = tokenizer.apply_chat_template(conversation=messages, tokenize=True, add_generation_prompt=True, return_tensors='pt')
output_ids = model.generate(input_ids.to('cuda'))
response = tokenizer.decode(output_ids[0][input_ids.shape[1]:], skip_special_tokens=True)

# Model response: "Hello! How can I assist you today?"
print(response)

The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.


Hello! How can I help you today?


In [ ]:
def formatting_prompts_func(example):
    output_texts = []
    for i in range(len(example['instruction'])):
        text = f"### Question: {example['instruction'][i]}\n ### Answer: {example['output'][i]}"
        output_texts.append(text)
    return output_texts

response_template = " ### Answer:"
collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)